In [ ]:
print(f"\n** RAG implementation of Llama2 using pdf **")

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()
#OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
MODEL = "llama2"

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

#model.invoke("how to impress my boyfriend who likes business and build wealth and family ?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
#chain.invoke("Tell me the joke")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Arun_Resume.pdf") # I used my Resume, change the pdfs to your need
pages = loader.load_and_split()
#pages


In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
chain = prompt| model | parser  
chain.input_schema.schema()

In [ ]:
chain.invoke(
 {
    "context":"My name is Aaron.",
    "question":"What's my name ?"
 })

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("Cache")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,#array of documents
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "where did he study?",
    "Whats the name ?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

In [ ]:
chain.batch([{"question": q} for q in questions])